# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

In this lab you will: 

- Use ANOVA for testing multiple pairwise comparisons 
- Interpret results of an ANOVA and compare them to a t-test

In [3]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

## Load the data

Start by loading in the data stored in the file `'ToothGrowth.csv'`: 

In [7]:
# Your code here
df = pd.read_csv('ToothGrowth.csv')
df.sample(5)

,len,supp,dose
5,10.0,VC,0.5
45,25.2,OJ,1.0
3,5.8,VC,0.5
0,4.2,VC,0.5
53,24.5,OJ,2.0


## Generate the ANOVA table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage:  

In [8]:
# Your code here
formula = 'len ~ C(supp) + dose'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

               sum_sq    df           F        PR(>F)
C(supp)    205.350000   1.0   11.446768  1.300662e-03
dose      2224.304298   1.0  123.988774  6.313519e-16
Residual  1022.555036  57.0         NaN           NaN


## Interpret the output

Make a brief comment regarding the statistics and the effect of supplement and dosage on tooth length: 

In [9]:
# Your comment here
# Both supplement and dose has significant effect
# Dose has more effect than supplemant

## Compare to t-tests

Now that you've had a chance to generate an ANOVA table, its interesting to compare the results to those from the t-tests you were working with earlier. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterward, you'll conduct a t-test to compare the tooth length of these two different samples: 

In [13]:
# Your code here
df_VC = df[df.supp == 'VC']['len']
df_OJ = df[df.supp == 'OJ']['len']

Now run a t-test between these two groups and print the associated two-sided p-value: 

In [14]:
# Calculate the 2-sided p-value for a t-test comparing the two supplement groups
from scipy import stats
stats.ttest_ind(df_VC, df_OJ)

Ttest_indResult(statistic=-1.91526826869527, pvalue=0.06039337122412849)

## A 2-Category ANOVA F-test is equivalent to a 2-tailed t-test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [15]:
# Your code here; conduct an ANOVA F-test of the oj and vc supplement groups.

formula = 'len ~ C(supp)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

# Compare the p-value to that of the t-test above. 
# The p-value from t-test matched the t-value from ANOVA

# They should match (there may be a tiny fractional difference due to rounding errors in varying implementations)

               sum_sq    df         F    PR(>F)
C(supp)    205.350000   1.0  3.668253  0.060393
Residual  3246.859333  58.0       NaN       NaN


## Run multiple t-tests

While the 2-category ANOVA test is identical to a 2-tailed t-test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [34]:
dfg = df.groupby(['supp', 'dose'])
dfg.head(5)

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5
10,16.5,VC,1.0
11,16.5,VC,1.0
12,15.2,VC,1.0
13,17.3,VC,1.0
14,22.5,VC,1.0


In [25]:
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)

('OJ', 0.5)
('OJ', 1.0)
('OJ', 2.0)
('VC', 0.5)
('VC', 1.0)
('VC', 2.0)


While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2-sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [48]:
# Your code here; reuse your t-test code above to calculate the p-value for a 2-sided t-test
# for all combinations of the supplement-dose groups listed above. 
# (Since there isn't a control group, compare each group to every other group.)

from itertools import combinations

groups = df.groupby(['supp', 'dose'])['len']

for gr_a, gr_b in combinations(groups, 2):
    result = stats.ttest_ind(gr_a[1], gr_b[1])
    print(f"T-test for {gr_a[0]} & {gr_b[0]}: p-value = {result[1]}")
    

T-test for ('OJ', 0.5) & ('OJ', 1.0): p-value = 8.357559281443774e-05
T-test for ('OJ', 0.5) & ('OJ', 2.0): p-value = 3.4018585295016214e-07
T-test for ('OJ', 0.5) & ('VC', 0.5): p-value = 0.005303661339923052
T-test for ('OJ', 0.5) & ('VC', 1.0): p-value = 0.04223992429368205
T-test for ('OJ', 0.5) & ('VC', 2.0): p-value = 7.025409196997986e-06
T-test for ('OJ', 1.0) & ('OJ', 2.0): p-value = 0.03736279585664383
T-test for ('OJ', 1.0) & ('VC', 0.5): p-value = 1.3372624230559434e-08
T-test for ('OJ', 1.0) & ('VC', 1.0): p-value = 0.0007807261651774468
T-test for ('OJ', 1.0) & ('VC', 2.0): p-value = 0.09583711277517494
T-test for ('OJ', 2.0) & ('VC', 0.5): p-value = 1.3381068810881244e-11
T-test for ('OJ', 2.0) & ('VC', 1.0): p-value = 2.3131084633597503e-07
T-test for ('OJ', 2.0) & ('VC', 2.0): p-value = 0.9637097790041267
T-test for ('VC', 0.5) & ('VC', 1.0): p-value = 6.492264598157612e-07
T-test for ('VC', 0.5) & ('VC', 2.0): p-value = 4.957285658438862e-09
T-test for ('VC', 1.0) & (

## Summary

In this lesson, you implemented the ANOVA technique to generalize testing methods to multiple groups and factors.